# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']
pprint(establishments.find_one())

{'AddressLine1': '16 The Stade',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Mariner',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.912144538092,
 'FHRSID': 289560,
 'LocalAuthorityBusinessID': 'PI/000039927',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6AB',
 'RatingDate': '2019-03-19T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64701888dc4f69d91987923e'),
 'geocode': {'latitude': 51.08084, 'longitude': 1.188537},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/289560',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene':20}
fields = {'BusinessName'}
# Use count_documents to display the number of documents in the result
results = establishments.find(query,fields)
print(establishments.count_documents(query))
# Display the first document in the results using pprint
pprint(results[0])


41
{'BusinessName': 'The Chase Rest Home',
 '_id': ObjectId('64701889dc4f69d91987ad75')}


In [7]:
import pandas as pd
# Convert the result to a Pandas DataFrame
df=pd.DataFrame({"Business Name":[entry['BusinessName'] for entry in results]})

# Display the number of rows in the DataFrame
print(df.count(axis=0))
# Display the first 10 rows of the DataFrame
df.head(10)

Business Name    41
dtype: int64


,Business Name
0,The Chase Rest Home
1,Brenalwood
2,Melrose Hotel
3,Golden Palace
4,Seaford Pizza
5,Ashby's Butchers
6,South Sea Express Cuisine
7,Golden Palace
8,The Tulip Tree
9,F & S


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName':{'$regex':'London'},'RatingValue':{'$gte':4}}
fields = {'BusinessName'}
# Use count_documents to display the number of documents in the result
results = establishments.find(query,fields)
print(establishments.count_documents(query))
# Display the first document in the results using pprint
pprint(results[0])

33
{'BusinessName': "Charlie's", '_id': ObjectId('6470188adc4f69d91987c78f')}


In [9]:
# Convert the result to a Pandas DataFrame
df=pd.DataFrame({"Business Name":[entry['BusinessName'] for entry in results]})
# Display the number of rows in the DataFrame
print(df.count(axis=0))
# Display the first 10 rows of the DataFrame
df.head(10)

Business Name    33
dtype: int64


,Business Name
0,Charlie's
1,Mv City Cruises Erasmus
2,Benfleet Motor Yacht Club
3,Coombs Catering t/a The Lock and Key
4,Tilbury Seafarers Centre
5,Mv Valulla
6,Tereza Joanne
7,Brick Lane Brews
8,City Bar & Grill
9,WH Smith


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [10]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
longitude = establishments.find_one({"BusinessName":"Penang Flavours"},{'geocode.longitude'})['geocode']['longitude']
latitude = establishments.find_one({"BusinessName":"Penang Flavours"},{'geocode.latitude'})['geocode']['latitude']
print(latitude)
print(longitude)
blat=latitude-degree_search
tlat=latitude+degree_search
blon=longitude-degree_search
tlon=longitude+degree_search
print(blat,tlat,blon,tlon)
fields = {'BusinessName','geocode','scores.Hygiene'}
query = {'scores.Hygiene':{'$gte':0},'geocode.longitude':{'$gte':blon, '$lte':tlon},'geocode.latitude':{'$gte':blat, '$lte':tlat}}
sort =  [('scores.Hygiene',-1)]
limit = 5

results=establishments.find(query,fields).sort(sort)
# Print the results
for i in range(establishments.count_documents(query)):
    pprint(results[i])


51.490142
0.08384
51.480142 51.500142 0.07384 0.09383999999999999
{'BusinessName': 'The Cakery',
 '_id': ObjectId('6470188bdc4f69d919880655'),
 'geocode': {'latitude': 51.4875866, 'longitude': 0.0910684},
 'scores': {'Hygiene': 15}}
{'BusinessName': 'The Cakery',
 '_id': ObjectId('6470188bdc4f69d919880655'),
 'geocode': {'latitude': 51.4875866, 'longitude': 0.0910684},
 'scores': {'Hygiene': 15}}
{'BusinessName': 'Capricorn Members Club',
 '_id': ObjectId('6470188bdc4f69d91988063a'),
 'geocode': {'latitude': 51.4873098, 'longitude': 0.0909549},
 'scores': {'Hygiene': 10}}
{'BusinessName': 'Capricorn Members Club',
 '_id': ObjectId('6470188bdc4f69d91988063a'),
 'geocode': {'latitude': 51.4873098, 'longitude': 0.0909549},
 'scores': {'Hygiene': 10}}
{'BusinessName': 'Apna Pind',
 '_id': ObjectId('6470188bdc4f69d91988067a'),
 'geocode': {'latitude': 51.4876894, 'longitude': 0.0905233},
 'scores': {'Hygiene': 10}}
{'BusinessName': 'Capricorn Members Club',
 '_id': ObjectId('6470188bdc4f69d

In [11]:
# Convert result to Pandas DataFrame
ls=[[entry['BusinessName'],entry['geocode']['latitude'],entry['geocode']['longitude'],entry['scores']['Hygiene']] for entry in results]
df=pd.DataFrame(ls, columns=["Business Name","Latitude", "Longitude", "Hygiene"])
df.head(5)

,Business Name,Latitude,Longitude,Hygiene
0,The Cakery,51.487587,0.091068,15
1,Ebute Metta Joint,51.488374,0.089750,15
2,Zehra's Cafe,51.486650,0.091141,10
3,Capricorn Members Club,51.487310,0.090955,10
4,Apna Pind,51.487689,0.090523,10


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [15]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match_query = {'$match': {'scores.Hygiene':0}}
group_query = {'$group': {'_id': '$LocalAuthorityName', 'count': { '$sum': 1 }}}
sort_values = {'$sort': { 'count': -1}}
pipeline = [match_query, group_query, sort_values]
results = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
print(len(results))
# Print the first 10 results
pprint(results[0:10])

55
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [16]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Row number: ", len(result_df))
# Display the first 10 rows of the DataFrame
result_df.head(10)

Row number:  55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
